Copyright (C) Microsoft Corporation.

# Deploy model as a Web Service

**Important**: Make sure the kernel is set to "Your project name local" which can be done from the *Kernel* menu under *Change kernel*. 

The web services can be deployed locally or to a cluster. Best practice is to start with a local deployment, validate that your model and code work, and then deploy to a cluster for production scale use. In this notebook, you will first download your operationalization files, register  machine learning compute resource provider in your subscription and create a model management account.

Next, **If you have Docker engine running locally**, you can continue with the instuctions for [local environment setup](#Local-environment-setup) section and follow the rest of the instructions to deploy your models locally. **If you are not running Docker locally**, then you will need to skip the local environment setup section and follow the steps for [cluster environment setup](#Cluster-environment-setup) and the rest of the instructions to deploy your web service to a cluster.

In [1]:
import os
import base64
import requests
import json
import zipfile
from azure.storage.blob import BlockBlobService

## Download operationalization files

First, you will download the zip file you uploaded to blob in the earlier notebook and extract it to get the model and scoring script. Locate your storage account name and key as described in the Getting Started document of this tutorial and provide your values in the following cell where indicated.

In [1]:
ACCOUNT_NAME = 'Your Storage Account Name Here!!'
ACCOUNT_KEY = 'Your Storage Account Key Here!!'
CONTAINER_NAME = "deploy"
ZIP_FILE = 'o16n.zip'
DESTINATION_FILE = ZIP_FILE

Download and extract the files to the current working directory.

In [4]:
block_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)
block_blob_service.get_blob_to_path(container_name=CONTAINER_NAME, blob_name=ZIP_FILE, file_path=DESTINATION_FILE)
zip = zipfile.ZipFile(ZIP_FILE)
zip.extractall()

## Register resource provider

First, make sure the Azure resource provider Microsoft.MachineLearningCompute is registered in your subscription.This is a one time task per subscription.

In [5]:
!az provider register -n Microsoft.MachineLearningCompute

Make sure you see "registrationState": "Registered" in the next cell output before you proceed to the next steps.

In [6]:
!az provider show -n Microsoft.MachineLearningCompute

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/providers/Microsoft.MachineLearningCompute",
  "namespace": "Microsoft.MachineLearningCompute",
  "registrationState": "Registered",
  "resourceTypes": [
    {
      "aliases": null,
      "apiVersions": [
        "2017-08-01-preview",
        "2017-06-01-preview"
      ],
      "locations": [
        "East US 2",
        "West Central US",
        "Australia East",
        "West Europe",
        "Southeast Asia"
      ],
      "properties": null,
      "resourceType": "operationalizationClusters"
    },
    {
      "aliases": null,
      "apiVersions": [
        "2017-08-01-preview",
        "2017-06-01-preview"
      ],
      "locations": [
        "East US 2"
      ],
      "properties": null,
      "resourceType": "operations"
    },
    {
      "aliases": null,
      "apiVersions": [
        "2017-08-01-preview",
        "2017-06-01-preview"
      ],
      "locations": [
        "East US 2"
      ],
      "properties":

## Create names for deployment

To deploy and test your web services, provide an Azure region and a deployment name in the following cell where indicated.

In [5]:
LOCATION = 'Pick an Azure Region Here!! e.g. eastus2'
DEPLOY_NAME = 'Pick a Deployment Name Here!!'

## Create a model management account

You need to create a [Model Management Account](https://docs.microsoft.com/en-us/azure/machine-learning/preview/model-management-overview) to deploy your models. You need to do this once per subscription, and can reuse the same account in multiple deployments.

In [6]:
MM_ACCOUNT = DEPLOY_NAME + 'mmacc'     
MM_ACCOUNT_RESOURCE_GROUP = MM_ACCOUNT + 'rg'

Create a resource group for model management account which will be used for both local and cluster deployments.

In [9]:
!az group create --l $LOCATION --name $MM_ACCOUNT_RESOURCE_GROUP

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/imagetutmmaccrg",
  "location": "eastus2",
  "managedBy": null,
  "name": "imagetutmmaccrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Now, create the model management account in the resource group created above.

In [10]:
!az ml account modelmanagement create -l $LOCATION -n $MM_ACCOUNT -g $MM_ACCOUNT_RESOURCE_GROUP

{
  "created_on": "2017-11-03T17:51:11.9226449999999999Z",
  "description": "",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/imagetutmmaccrg/providers/Microsoft.MachineLearningModelManagement/accounts/imagetutmmacc",
  "location": "eastus2",
  "model_management_swagger_location": "https://eastus2.modelmanagement.azureml.net/api/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/imagetutmmaccrg/accounts/imagetutmmacc/swagger.json?api-version=2017-09-01-preview",
  "modified_on": "2017-11-03T17:51:11.9226449999999999Z",
  "name": "imagetutmmacc",
  "resource_group": "imagetutmmaccrg",
  "sku": {
    "capacity": 1,
    "name": "S1"
  },
  "subscription": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "tags": {},
  "type": "Microsoft.MachineLearningModelManagement/accounts"
}


Verify that model management account is set.

In [11]:
!az ml account modelmanagement show

{
  "created_on": "2017-11-03T17:51:11.9226449999999999Z",
  "description": "",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/imagetutmmaccrg/providers/Microsoft.MachineLearningModelManagement/accounts/imagetutmmacc",
  "location": "eastus2",
  "model_management_swagger_location": "https://eastus2.modelmanagement.azureml.net/api/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/imagetutmmaccrg/accounts/imagetutmmacc/swagger.json?api-version=2017-09-01-preview",
  "modified_on": "2017-11-03T17:51:11.9226449999999999Z",
  "name": "imagetutmmacc",
  "resource_group": "imagetutmmaccrg",
  "sku": {
    "capacity": 1,
    "name": "S1"
  },
  "subscription": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "tags": {},
  "type": "Microsoft.MachineLearningModelManagement/accounts"
}


## Local environment setup

In the following steps, you will deploy and test your web service on the local machine running Docker. If you are not running Docker locally, please jump to [the cluster environment setup](#Cluster-environment-setup) section.

In [14]:
LOCAL_ENV_NAME = DEPLOY_NAME + 'local'+ 'env'  
LOCAL_RESOURCE_GROUP = LOCAL_ENV_NAME + 'rg'

Set up a local environment using the following command which creates the following resources in your subscription:
- A resource group with "rg" added at the end of the environment name you picked
- A storage account
- An Azure Container Registry (ACR)
- An Application insights account

In [15]:
!az ml env setup -l $LOCATION -n $LOCAL_ENV_NAME  --yes

Creating resource group imagetutlocalenvrg
Provisioning compute resources...
Resource creation submitted successfully.
To see more information for your environment, run:
  az ml env show -g imagetutlocalenvrg -n imagetutlocalenv
You can set the new environment as your target context using:
  az ml env set -g imagetutlocalenvrg -n imagetutlocalenv


You can check the information on the environment with the following. You will need to wait until "Provisioning State" is set to "Succeeded" to continue with the next steps.

In [20]:
!az ml env show -g $LOCAL_RESOURCE_GROUP -n $LOCAL_ENV_NAME

{
  "Cluster Name": "imagetutlocalenv",
  "Cluster Size": "N/A",
  "Created On": "2017-11-06T14:34:11.309Z",
  "Location": "eastus2",
  "Provisioning State": "Succeeded",
  "Resource Group": "imagetutlocalenvrg",
  "Subscription": "edf507a2-6235-46c5-b560-fd463ba2e771"
}


Now, set the environment to be used.

In [21]:
!az ml env set -g $LOCAL_RESOURCE_GROUP -n $LOCAL_ENV_NAME

Compute set to imagetutlocalenv.


Check if your environment is set.

In [22]:
!az ml env show

{
  "Cluster Name": "imagetutlocalenv",
  "Cluster Size": "N/A",
  "Created On": "2017-11-06T14:34:11.309Z",
  "Current Mode": "local",
  "Location": "eastus2",
  "Provisioning State": "Succeeded",
  "Resource Group": "imagetutlocalenvrg",
  "Subscription": "edf507a2-6235-46c5-b560-fd463ba2e771"
}


### Deploy model locally

Now you can deploy your model with the following command.
```bash
az ml service create realtime --model-file [model file/folder path] -f [scoring file e.g. score.py] -n [your service name] -s [schema file e.g. service_schema.json] -r [runtime for the Docker container e.g. spark-py or python] -c [conda dependencies file for additional python packages]
```

First, set the service name.

In [24]:
LOCAL_SERVICE_NAME = DEPLOY_NAME + 'local' + 'srvc'

Create the real time service.

In [25]:
!az ml service create realtime --model-file lightgbm_classifier.model -f imgscore.py -n $LOCAL_SERVICE_NAME -r python -c aml_config/conda_dependencies_srvc.yml

 lightgbm_classifier.model
Successfully registered model
Id: 6328903bd0854d34808aea3832ab8299
More information: 'az ml model show -m 6328903bd0854d34808aea3832ab8299'
Creating new driver at C:\Users\fboylu\AppData\Local\Temp\tmp329rld32.py
 imgscore.py
Successfully created manifest
Id: 291cc2c7-6783-4294-89f4-540f2e79ee1c
More information: 'az ml manifest show -i 291cc2c7-6783-4294-89f4-540f2e79ee1c'
Creating image..................................................................................Done.
Image ID: b7416cdf-f3e3-44f9-821c-d7d21e5c8960
More details: 'az ml image show -i b7416cdf-f3e3-44f9-821c-d7d21e5c8960'
Usage information: 'az ml image usage -i b7416cdf-f3e3-44f9-821c-d7d21e5c8960'
[Local mode] Running docker container.
[Local mode] Pulling the image from mlcrpacrfcc24453077f.azurecr.io/imagetutlocalsrvc:6. This may take a few minutes, depending on your connection speed...
[Local mode] Pulling................................................................................

Check the state of the service.

In [26]:
!az ml service list realtime -o table

Name               Id                 UpdatedAt            State
-----------------  -----------------  -------------------  -------
imagetutlocalsrvc  imagetutlocalsrvc  2017-11-03T20:12:12  running


After the web service has been successfully deployed, you can use the following command to get the service URL and other details for calling the service endpoint. You will use the Scoring URL in the output below for calling the service in the following steps.

In [27]:
!az ml service usage realtime -i $LOCAL_SERVICE_NAME

Scoring URL:
    http://127.0.0.1:32771/score

Headers:
    Content-Type: application/json

Swagger URL:
    http://127.0.0.1:32771/swagger.json

Sample CLI command:
    Usage for cmd: az ml service run realtime -i imagetutlocalsrvc -d !! YOUR DATA HERE !!
    Usage for powershell: az ml service run realtime -i imagetutlocalsrvc --% -d !! YOUR DATA HERE !!

Sample CURL call:
    curl -X POST -H "Content-Type:application/json" --data !! YOUR DATA HERE !! http://127.0.0.1:32771/score

Get debug logs by calling:
    az ml service logs realtime -i imagetutlocalsrvc




Now, you will send a request to your real-time web service. Provide the scoring URL from the output above in the following cell.

In [28]:
url = 'Your Scoring URL Here!!'

You will now call the service with the sample image  you extracted from the operationalization folder. Encode the image into string representation and then serialize it into json format.

In [29]:
img_path = 'fail.0.jpg'
with open(img_path, 'rb') as file:
  encoded = base64.b64encode(file.read())
body = json.dumps("{}".format(encoded))

Use the following code to make a prediction.

In [30]:
headers = {'Content-Type':'application/json'}
response = requests.post(url, headers=headers, data=body)
prediction = json.loads(response.content.decode('ascii'))
prediction

[[0.00011646905475415675]]

Delete local deployment to avoid any conflicts with the cluster deployment. The service can be deleted with the following command which will delete the local docker image and the container.

In [31]:
!az ml service delete realtime -i $LOCAL_SERVICE_NAME

Service deleted.


Delete the environment.

In [23]:
!az ml env delete -n $LOCAL_ENV_NAME -g $LOCAL_RESOURCE_GROUP

Resource deletion successfully submitted.
Resources may take 1-2 minutes to be completely deprovisioned.


You can also delete the resource group you created for the local deployment.

In [24]:
!az group delete --name $LOCAL_RESOURCE_GROUP --yes

The following cell will display a WARNING and ERROR which is expected behaviour since we deleted the resource group which leads to desired outcome of unsetting the current environment to prepare it for cluster deployment in the next section.

In [25]:
!az ml env show

  az ml env list
To set an active environment, run:
  az ml env set -n <env_name> -g <env_group>
To see available subscriptions, run:
  az account show -o table
To set active accout, run:
  az account set -s <subscription_name>

ERROR: ResourceGroupNotFound: Resource group 'imagetutlocalenvrg' could not be found.


## Cluster environment setup 

Use Cluster deployment for high-scale production scenarios. It sets up an ACS cluster with Kubernetes as the orchestrator. The ACS cluster can be scaled out to handle larger throughput for your web service calls. First, we provide a name for the cluster environment.

In [7]:
CLUSTER_SERVICE_NAME = DEPLOY_NAME + 'clus' + 'srvc'
CLUSTER_ENV_NAME = DEPLOY_NAME + 'clus' + 'env'
CLUSTER_RESOURCE_GROUP = CLUSTER_ENV_NAME + 'rg'

Setup cluster environment using the following command which creates the following resources in your subscription:
- A resource group with "rg" added at the end of the environment name
- A storage account
- An Azure Container Registry (ACR)
- A Kubernetes deployment on an Azure Container Service (ACS) cluster
- An Application insights account

To deploy your web service to a production environment, first set up the environment using the following command:

In [8]:
!az ml env setup --cluster -l $LOCATION -n $CLUSTER_ENV_NAME  --yes

Creating resource group imagetutclusenvrg
waiting for AAD role to propagate.done
Provisioning compute resources...
Resource creation submitted successfully.
Resources may take 10-20 minutes to be completely provisioned.
To see if your environment is ready to use, run:
  az ml env show -g imagetutclusenvrg -n imagetutclusenv
Once your environment has successfully provisioned, you can set it as your target context using:
  az ml env set -g imagetutclusenvrg -n imagetutclusenv


You can check the information on the environment with the following. You will need to wait until "Provisioning State" is set to "Succeeded" to continue with the next steps.

In [15]:
!az ml env show -g $CLUSTER_RESOURCE_GROUP -n $CLUSTER_ENV_NAME

{
  "Cluster Name": "imagetutclusenv",
  "Cluster Size": 2,
  "Created On": "2017-11-06T21:08:38.636Z",
  "Location": "eastus2",
  "Provisioning State": "Succeeded",
  "Resource Group": "imagetutclusenvrg",
  "Subscription": "edf507a2-6235-46c5-b560-fd463ba2e771"
}


Next step is to set the cluster environment for deployment.

In [16]:
!az ml env set -g $CLUSTER_RESOURCE_GROUP -n $CLUSTER_ENV_NAME --disable-dashboard

Compute set to imagetutclusenv.


Check that environment is set to the cluster.

In [18]:
!az ml env show

{
  "Cluster Name": "imagetutclusenv",
  "Cluster Size": 2,
  "Created On": "2017-11-06T21:08:38.636Z",
  "Current Mode": "cluster",
  "Location": "eastus2",
  "Provisioning State": "Succeeded",
  "Resource Group": "imagetutclusenvrg",
  "Subscription": "edf507a2-6235-46c5-b560-fd463ba2e771"
}


Now, set the model management account for the environment.

In [33]:
!az ml account modelmanagement set -n $MM_ACCOUNT -g $MM_ACCOUNT_RESOURCE_GROUP

{
  "created_on": "2017-11-03T17:51:11.9226449999999999Z",
  "description": "",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/imagetutmmaccrg/providers/Microsoft.MachineLearningModelManagement/accounts/imagetutmmacc",
  "location": "eastus2",
  "model_management_swagger_location": "https://eastus2.modelmanagement.azureml.net/api/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/imagetutmmaccrg/accounts/imagetutmmacc/swagger.json?api-version=2017-09-01-preview",
  "modified_on": "2017-11-03T17:51:11.9226449999999999Z",
  "name": "imagetutmmacc",
  "resource_group": "imagetutmmaccrg",
  "sku": {
    "capacity": 1,
    "name": "S1"
  },
  "subscription": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "tags": {},
  "type": "Microsoft.MachineLearningModelManagement/accounts"
}


### Deploy model to cluster

Print your cluster service name  for creating the service in the next steps.

In [19]:
print('Your Cluster Service Name: ' + CLUSTER_SERVICE_NAME)

Your Cluster Service Name: imagetutclussrvc


You will need to change your directory to the current directory on Workbench CLI. Use the output in the next cell.

In [20]:
print('Your Working Directory: '+ os.getcwd())

Your Working Directory: C:\Users\fboylu\AppData\Local\Temp\azureml_runs\imagetutorial3_1510002303767


Now, go to the Azure Machine Learning Workbench CLI window and change your directory using:
```
cd [Your Working Directory Here!!]
```

Next step is to deploy your model to the cluster environment. However, since the command below is not available through Jupyter notebooks for cluster environment set up, we will use Azure Machine Learning Workbench CLI.

In [39]:
# This command is not available through Jupyter notebooks for cluster set up.
#!az ml service create realtime --model-file lightgbm_classifier.model -f imgscore.py -n $CLUSTER_SERVICE_NAME -r python -c aml_config/conda_dependencies_srvc.yml

Go to the Azure Machine Learning Workbench CLI window and issue the following command by providing your cluster service name printed earlier where indicated in the following command.
```bash
az ml service create realtime --model-file lightgbm_classifier.model -f imgscore.py -n [Your Cluster Service Name Here!!] -r python -c aml_config/conda_dependencies_srvc.yml
```
You can watch the steps of the deployment from the CLI output which can take up to 20 minutes. Once completed, return back to the notebook.

List the service you deployed.

In [41]:
!az ml service list realtime -o table

Name              Id                                                 UpdatedAt            State
----------------  -------------------------------------------------  -------------------  ---------
imagetutclussrvc  imagetutclussrvc.imagetutclusenv-47424707.eastus2  2017-11-06T16:51:30  Succeeded


Print your cluster service Id in order to use it in the following steps.

In [42]:
!az ml service list realtime | grep Id

    "Id": "imagetutclussrvc.imagetutclusenv-47424707.eastus2",


Provide your cluster service Id printed earlier in the cell below where indicated.

In [43]:
CLUSTER_SERVICE_ID = 'Your cluster service Id here!!'

In [44]:
!az ml service usage realtime -i $CLUSTER_SERVICE_ID

Scoring URL:
    http://40.79.80.172:80/api/v1/service/imagetutclussrvc/score

Headers:
    Content-Type: application/json
    Authorization: Bearer <key>
        (<key> can be found by running 'az ml service keys realtime -i imagetutclussrvc.imagetutclusenv-47424707.eastus2')

Swagger URL:
    http://40.79.80.172:80/api/v1/service/imagetutclussrvc/swagger.json

Sample CLI command:
    Usage for cmd: az ml service run realtime -i imagetutclussrvc.imagetutclusenv-47424707.eastus2 -d !! YOUR DATA HERE !!
    Usage for powershell: az ml service run realtime -i imagetutclussrvc.imagetutclusenv-47424707.eastus2 --% -d !! YOUR DATA HERE !!

Sample CURL call:
    curl -X POST -H "Content-Type:application/json" -H "Authorization:Bearer <key>" --data !! YOUR DATA HERE !! http://40.79.80.172:80/api/v1/service/imagetutclussrvc/score

Get debug logs by calling:
    az ml service logs realtime -i imagetutclussrvc.imagetutclusenv-47424707.eastus2

Get STDOUT/STDERR or Request/Response logs in App In

Now, you can send a request to your real-time web service. Provide the scoring URL from the output above in the following cell.

In [45]:
url = 'Your Scoring URL Here!!'

For cluster deployments, an api key needs to be provided. 

In [46]:
!az ml service keys realtime -i $CLUSTER_SERVICE_ID

PrimaryKey: 62bcbaaeac7e455eabd87177607dadee
SecondaryKey: 09fec15f3d124686894345e675d3e95b


Provide the "PrimaryKey" from the output above as your Api Key in the following cell where indicated.

In [47]:
api_key = 'Your Api Key Here!!'

You will now call the service with the sample image you extracted from the operationalization folder. Encode the image into string representation and then serialize it into json format.

In [49]:
import base64, requests, json
img_path = 'fail.0.jpg'
with open(img_path, 'rb') as file:
  encoded = base64.b64encode(file.read())
body = json.dumps("{}".format(encoded))

Use the following code to make a prediction.

In [50]:
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
response = requests.post(url, headers=headers, data=body)
prediction = json.loads(response.content.decode('ascii'))
prediction

[[0.00011646905475415675]]

The service can be deleted with the following command.

In [54]:
!az ml service delete realtime -i $CLUSTER_SERVICE_ID

Delete cluster environment.

In [51]:
!az ml env delete -n $CLUSTER_ENV_NAME -g $CLUSTER_RESOURCE_GROUP

Resource deletion successfully submitted.
Resources may take 1-2 minutes to be completely deprovisioned.


Delete the resource group to clean up.

In [55]:
!az group delete --name $CLUSTER_RESOURCE_GROUP --yes

Check that current environment is unset.

In [56]:
!az ml env show

  az ml env list
To set an active environment, run:
  az ml env set -n <env_name> -g <env_group>
To see available subscriptions, run:
  az account show -o table
To set active accout, run:
  az account set -s <subscription_name>

ERROR: ResourceGroupNotFound: Resource group 'imagetutclusenvrg' could not be found.
